In [ ]:
import os
import warnings
from dotenv import load_dotenv
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import AzureChatOpenAI
from langchain.tools import tool
from langchain.tools import WikipediaQueryRun
from langchain import PromptTemplate
from langchain_community.utilities import WikipediaAPIWrapper
from pandas import pandas

warnings.filterwarnings("ignore")

# Load environment variables from a .env file  
load_dotenv()

In [ ]:
# Create a Language Model (LLM) client using the Azure OpenAI API  
# Set the temperature to zero to avoid randomness in the output  
llm_model = 'gpt-3.5-turbo'
llm = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    model=llm_model,
    temperature=0
)

In [ ]:
# Define the first custom tool  
# This tool loads a CSV file with a list of all cities in the Netherlands and returns a random row 
@tool
def dutch_cities(query: str) -> str:
    """Useful if you need a name of a Dutch city"""
    return pandas.read_csv('dutch_cities.csv').sample().iloc[0][0]

In [ ]:
# Example usage of the dutch_cities tool 
dutch_cities.run('')
# Example output: 'Hartwerd'

In [ ]:
# Override the built-in Wikipedia tool, setting the language to Dutch (NL)  
tool_wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(lang="nl"))

In [ ]:
# Example usage of the Wikipedia tool  
tool_wikipedia.run('Leek')
# Example output:  
# "Page: Leek (gemeente)  
# Summary: Leek () (Gronings: (De) Laik of De Lieke, Fries: De Like) is een voormalige gemeente in Noord-Nederland, in het Westerkwartier van de provincie Groningen. De gemeente besloeg een oppervlakte van 64 km², waarvan 0,63 km² water en telde 19.680 inwoners (31 mei 2018). De gemeente Leek is op 1 januari 2019 opgegaan in de gemeente Westerkwartier, samen met de gemeentes Marum, Grootegast, en Zuidhorn.  
#   
# Page: Leek (christendom)  
# Summary: Met leek (van het Grieks: λαϊκός, 'van het volk') wordt in het christendom in de regel een gelovige aangeduid die geen wijding of benoeming heeft ontvangen tot een kerkelijk ambt. Ook pastoraal werkers zijn leken."  

In [ ]:
# Define a custom tool to extract a trivia fact from a large text 
@tool
def extract_trivia_from_text(input: str) -> str:
    """Useful when you need to extract one fact from an informative text."""
    return llm.predict(PromptTemplate(
    template="""Given the following text `{input}`.

      Choose one trivia from the text, and summerize it in one sentence. Avoid trivia that contains numbers.
      Your answer should be in Dutch.
    """,
    input_variables=["input"],
).format(input = input))

In [ ]:
# Example usage of the extract_trivia_from_text tool  
text = "Spierdijk is een dorp in de gemeente Koggenland, in de regio West-Friesland in de Nederlandse provincie Noord-Holland. Het dorp heeft 1.500 inwoners (2023). Onder de oudere generatie wordt nog Westfries dialect gesproken, maar de meeste inwoners spreken Standaardnederlands. is hiervan gemaakt."
extract_trivia_from_text.run(text)
# Example output:  
# 'Spierdijk is een dorp in de gemeente Koggenland waar onder de oudere generatie nog Westfries dialect wordt gesproken.'

In [ ]:
#Define a custom tool to write a newspaper article in Dutch about the input text
@tool
def write_newspaper_article(input: str) -> str:
    """Useful when you need to write an newspaper article"""
    return llm.predict(PromptTemplate(
    template="""Given the following text `{input}`.

      Write an cynical newspaper article about the input. Use 100 words.
      Your answer should be in Dutch.
    """,
    input_variables=["input"],
).format(input = input))

In [ ]:
# Example usage of the write_newspaper_article tool 
text = 'Spierdijk is een dorp in de gemeente Koggenland waar onder de oudere generatie nog Westfries dialect wordt gesproken.'
write_newspaper_article.run(text)
# Example output:  
# 'Spierdijk, het dorp waar de tijd lijkt te hebben stilgestaan. Terwijl de rest van de wereld vooruitgaat, blijft de oudere generatie hier vasthouden aan het verleden door het spreken van het Westfries dialect. Een taal die inmiddels door de meeste mensen als achterhaald wordt beschouwd. Het is bijna komisch om te zien hoe deze inwoners krampachtig vasthouden aan hun tradities, terwijl de rest van de wereld moderniseert. Misschien wordt het tijd dat Spierdijk zich aanpast aan de huidige tijd, in plaats van vast te blijven houden aan oude gewoontes.'

In [ ]:
# Chain all the tools together and use the LLM to perform complex tasks
tools = [dutch_cities, tool_wikipedia, extract_trivia_from_text, write_newspaper_article]

# Initialize an agent with the tools and the LLM
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [ ]:
# Print the actual prompt that is sent over the line
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

In [ ]:
# Define a question for the agent to solve using the chained tools
question = "Pak een willekeurige plaats uit Nederland. Vindt informatie over die plaats. En haal uit die informatie een leuke weetje en schrijf daar een column over."

# Get the result from the agent
result = agent.invoke({"input": question})

example output:
````plaintext
> Entering new AgentExecutor chain...
Question: Select a random place in the Netherlands. Find information about that place and extract an interesting fact from that information. Then write a column about that fact.

Thought: I can use the dutch_cities tool to select a random place in the Netherlands, then use the wikipedia tool to find information about that place. After that, I can use the extract_trivia_from_text tool to extract an interesting fact, and finally use the write_newspaper_article tool to write a column about that fact.

Action:
```
{
  "action": "dutch_cities",
  "action_input": "random"
}
```


Observation: Langerak
Thought:I will now use the Wikipedia tool to find information about the city of Langerak in the Netherlands. Then, I can use the extract_trivia_from_text tool to extract an interesting fact about Langerak, and finally use the write_newspaper_article tool to write a column about that fact.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Langerak, Netherlands"
}
```
Observation: Page: Vlag van Langerak
Summary: De vlag van Langerak is op onbekende datum vastgesteld als de gemeentevlag van de Zuid-Hollandse gemeente Langerak. De vlag bestaat uit twee banen van gelijke hoogte in de kleuren rood en geel. In het rode baan is links een geel kanton opgenomen met het gemeentewapen.
De vlag is nooit officieel vastgesteld. De vlag is gebaseerd op de defileervlag die in 1938 door vertegenwoordigers van de gemeente werd gedragen tijdens het defilé in Amsterdam ter gelegenheid van het veertigjarig regeringsjubileum van koningin Wilhelmina. De vlag bestaat uit twee banen van rood en geel (destijds aangewezen als provinciekleuren voor Zuid-Holland), met op het kanton het beeld van het gemeentelijk wapenschild.
Op 1 januari 1986 ging het grootste deel van de gemeente, waaronder de plaats Langerak, over naar de nieuw opgerichte gemeente Graafstroom. De vlag van Langerak is daardoor komen te vervallen. Sinds 1 januari 2019 valt Langerak onder de gemeente Molenlanden.

Page: Vlag van Numansdorp
Summary: De vlag van Numansdorp is op 19 februari 1970 vastgesteld als de gemeentevlag van de Zuid-Hollandse gemeente Numansdorp. De vlag wordt als volgt omschreven:

Drie golvende banen, waarvan de hoogten zich verhouden als 7:6:7, wit, blauw en wit, met drie schuinkruisjes, aan de broeking geplaatst op het midden van de hoogte van iedere baan, van het een in het ander.
De vlag is ontleend aan het gemeentewapen.
Op 1 januari 1984 is Numansdorp opgegaan in de gemeente Cromstrijen. De vlag is daardoor als gemeentevlag komen te vervallen. Op 1 januari 2019 is Cromstrijen opgegaan in de gemeente Hoeksche Waard.

Page: Vlag van Groot-Ammers
Summary: De vlag van Groot-Ammers is op onbekende datum vastgesteld als de officiële gemeentelijke vlag van de Zuid-Hollandse gemeente Groot-Ammers. De vlag wordt als volgt beschreven:

Geel, met op het midden een zwarte achtpuntige ster.
De vlag toont hetzelfde beeld als het gemeentewapen.
Op 1 januari 1986 is Groot-Ammers samen met Nieuwpoort, Langerak en Streefkerk opgegaan in de gemeente Liesveld. De vlag is daardoor als gemeentevlag komen te vervallen. Op 1 januari 2013 is Liesveld opgegaan in de fusiegemeente Molenwaard, die sinds 1 januari 2019 deel uitmaakt van de gemeente Molenlanden.
Thought:It seems that the information retrieved is about the flags of different municipalities in the Netherlands, including Langerak. I will use the extract_trivia_from_text tool to extract an interesting fact about Langerak from the information provided. Then, I can use the write_newspaper_article tool to write a column about that fact.

Action:
```
{
  "action": "extract_trivia_from_text",
  "action_input": "De vlag van Langerak is op onbekende datum vastgesteld als de gemeentevlag van de Zuid-Hollandse gemeente Langerak. De vlag bestaat uit twee banen van gelijke hoogte in de kleuren rood en geel. In het rode baan is links een geel kanton opgenomen met het gemeentewapen. De vlag is nooit officieel vastgesteld. De vlag is gebaseerd op de defileervlag die in 1938 door vertegenwoordigers van de gemeente werd gedragen tijdens het defilé in Amsterdam ter gelegenheid van het veertigjarig regeringsjubileum van koningin Wilhelmina. De vlag bestaat uit twee banen van rood en geel (destijds aangewezen als provinciekleuren voor Zuid-Holland), met op het kanton het beeld van het gemeentelijk wapenschild. Op 1 januari 1986 ging het grootste deel van de gemeente, waaronder de plaats Langerak, over naar de nieuw opgerichte gemeente Graafstroom. De vlag van Langerak is daardoor komen te vervallen. Sinds 1 januari 2019 valt Langerak onder de gemeente Molenlanden."
}
```


Observation: De vlag van Langerak is gebaseerd op de defileervlag die in 1938 werd gedragen ter gelegenheid van het veertigjarig regeringsjubileum van koningin Wilhelmina.
Thought:I have extracted an interesting fact about Langerak: its flag is based on the defileervlag that was carried in 1938 to celebrate the fortieth anniversary of Queen Wilhelmina's reign. Now, I will use the write_newspaper_article tool to write a column about this fact.

Action:
```
{
  "action": "write_newspaper_article",
  "action_input": "The flag of Langerak, a small town in the Netherlands, holds a unique historical significance. It is based on the defileervlag that was carried in 1938 to celebrate the fortieth anniversary of Queen Wilhelmina's reign. This flag, with two bands of red and yellow, was never officially established but was used during the defilé in Amsterdam. The image of the municipal coat of arms was also included on the flag. This historical connection to a significant royal event makes the flag of Langerak a symbol of local pride and heritage. The town of Langerak, now part of the municipality of Molenlanden, continues to carry this historical legacy through its flag."
}
```



Observation: De vlag van Langerak, een klein stadje in Nederland, wordt geprezen om zijn "unieke historische betekenis". Maar laten we eerlijk zijn, het is gewoon een vlag die gebaseerd is op een vlag die in 1938 werd gebruikt om de veertigste verjaardag van koningin Wilhelmina's heerschappij te vieren. Het werd nooit officieel vastgesteld en was slechts in gebruik tijdens een defilé in Amsterdam. Laten we niet vergeten dat het ook het wapen van de stad bevat. Dus laten we ophouden met doen alsof dit een enorm historisch symbool is. Het is gewoon een vlag met een twijfelachtige connectie met een koninklijk evenement.
Thought:I now know the final answer

Final Answer: The flag of Langerak, a small town in the Netherlands, is based on a flag that was used in 1938 to celebrate the fortieth anniversary of Queen Wilhelmina's reign. This historical connection makes the flag of Langerak a symbol of local pride and heritage.

> Finished chain.
````